# "지하철 이용승객 분석" EDA 프로젝트(1)
- 이번 프로젝트에서는 EDA가 무엇이고 , 실습을 통해 EDA 하는 방법을 배워보겠습니다.
- EDA(Exploratory Data Analysis 탐색적 데이터 분석)는 데이터를 이해하는 과정이며
- 통계적 요약, 분포 파악 및 시각화 등의 기법을 통해 직관적으로 데이터 특성 파악 할수 있습니다.
- 결국, EDA를 통해 데이터에서 트랜드 분석, 인사이트 도출하게 됩니다.


## 실습 데이터
- 지하철 승하자 이용객 데이터 : 2019.01.01 ~ 2019.06.30
  - 서울열린데이터광장 :  http://data.seoul.go.kr/dataList/OA-12914/A/1/datasetView.do
- 지하철 노선 정보 데이터

## EDA 실습을 통해 해결해야 할 질문들

- 2019.01~06 중에 언제 지하철을 가장 많이 이용했을까? (기준: 승하자총승객수)
- 1월~6월중에 5월에 지하철 승객수가 많다? (기준 :승하자총승객수)
- 요일중에서 목요일에 지하철 승객수가 많다? (기준 :승하자총승객수)
- 일자별(월일별) 승하차승객수 그래프 그려 볼까요? (기준 :승하자총승객수)
- 가장 승객이 많이 타는 승차역은?
- 노선별로 역별/요일별 승차승객수를 비교해 볼수 있을까? (1~9호선, 역별/요일별 heatmap)
- 1호선에서 가장 하자를 많이 하는 역은? (groupby)
- 2호선중에서 어느 역에서 승차가 가장 많이 발생할까? (Folium 역 표시)

## 해당 Jupyter Notebook에서 실습 내용
- 필요한 라이브러리 가져오기
- 필요한 파일 불러오고 병합하기(read_csv, merge)
- Feature Engineering : 요일, 연월, 월일, 승하자총승객수 컬럼 추가
- 데이터 저장하기

<br>

# 1. 필요한 라이브러리 가져오기

In [6]:
# 필요한 라이브러리를 불러옵니다.

import os # 폴더/파일을 관리하는 os 라이브러리
import pandas as pd
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# 파일이 저장된 Google Drive 경로 설정
drive_dir_path = '/content/drive/MyDrive/AI+X/'

# 해당 경로의 파일 목록 확인
files = os.listdir(drive_dir_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2 raw 파일 불러오기
- 열차/노선/지하철역별 승하자 고객수
- 19년 상반기(19년 1월 ~ 19년 6월)

In [15]:
# 작업할 여러개의 파일 중 하나의 파일을 불러옵니다.

if files:
    file = files[0] # 첫 번째 파일 선택
    filePath = os.path.join(drive_dir_path, file) # 운영체제 호환성을 맞추기 위해 넣기!

    # 선택한 파일을 불러옵니다.
    raw = pd.read_csv(filePath)
    print("파일을 성공적으로 불러왔습니다.")
    display(raw.head())
else:
    print("Google Drive 경로에 파일이 없습니다.")

파일을 성공적으로 불러왔습니다.


,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20190101,2호선,을지로4가,3862.0,3728.0,20190104
1,20190101,3호선,을지로3가,8104.0,7554.0,20190104
2,20190101,2호선,을지로입구,22478.0,21330.0,20190104
3,20190101,2호선,시청,8381.0,6049.0,20190104
4,20190101,6호선,동묘앞,8045.0,8504.0,20190104


In [18]:
# info() 이용해 데이터 구조를 살펴볼수 있습니다.
# 데이터 크기들, Non Null 개수, 데이터 타입들, 메모리 사용량 등
raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18334 entries, 0 to 18333
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   사용일자    18334 non-null  int64  
 1   노선명     18334 non-null  object 
 2   역명      18334 non-null  object 
 3   승차총승객수  18334 non-null  float64
 4   하차총승객수  18334 non-null  float64
 5   등록일자    18334 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 859.5+ KB


## 참고) 판다스로 파일 읽어오기

### 데이터 파일 읽기 : read_excel / read_csv
- pd.read_excel('파일경로+파일명.xlsx')
- pd.read_csv('파일경로+파일명.csv', encoding='utf-8')
- pd.read_csv('파일경로+파일명.csv', encoding='cp949') # MS 엑셀에 저장된 경우

In [22]:
# 이번에 Google Drive에서 특정 2개 파일을 불러와서 합치겠습니다.

# 불러올 두 개의 파일 이름 지정
file1_name = 'CARD_SUBWAY_MONTH_201901.csv'
file2_name = 'CARD_SUBWAY_MONTH_201902.csv'

# 파일의 전체 경로 생성
filePath1 = os.path.join(drive_dir_path, file1_name)
filePath2 = os.path.join(drive_dir_path, file2_name)

# 각 파일 불러오기
raw1 = pd.read_csv(filePath1)
raw2 = pd.read_csv(filePath2)

# 불러온 두 데이터프레임을 세로로 병합 (axis=0)
# ignore_index=True 옵션으로 새로운 인덱스 생성 -> 중복된 숫자를 가지지 않도록 하기!
raw = pd.concat([raw1, raw2], axis=0, ignore_index=True)

print(f"'{file1_name}'와 '{file2_name}' 파일을 성공적으로 병합했습니다.")
display(raw.head())
display(raw.tail())

In [ ]:
raw

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20190101,2호선,을지로4가,3862.0,3728.0,20190104
1,20190101,3호선,을지로3가,8104.0,7554.0,20190104
2,20190101,2호선,을지로입구,22478.0,21330.0,20190104
3,20190101,2호선,시청,8381.0,6049.0,20190104
4,20190101,6호선,동묘앞,8045.0,8504.0,20190104
...,...,...,...,...,...,...
16538,20190228,2호선,을지로4가,6128.0,7087.0,20190303
16539,20190228,2호선,동대문역사문화공원,3387.0,4061.0,20190303
16540,20190228,5호선,청구,4920.0,4474.0,20190303
16541,20190228,1호선,가산디지털단지,51729.0,52043.0,20190303


---

# Q) 폴더에 있는 모든 파일을 불러와서 병합할수 있을까?

In [ ]:
# 폴더/파일을 관리하는 os 라이브러리를 불러오기 -> 이미 위에서 했지만 한번더!

import os


In [59]:
# os.listdir()을 이용해 Google Drive 폴더에 있는 파일 리스트를 살펴봅니다.

print(f"'{drive_dir_path}' 경로의 파일 목록:")
files_in_drive_folder = os.listdir(drive_dir_path) # 이미 지정해둔 경로 읽어오기

# 해당 경로에 있는 파일 리스트 살펴보기
files_in_drive_folder

'/content/drive/MyDrive/AI+X/' 경로의 파일 목록:


['CARD_SUBWAY_MONTH_201901.csv',
 'CARD_SUBWAY_MONTH_201902.csv',
 'CARD_SUBWAY_MONTH_201903.csv',
 'CARD_SUBWAY_MONTH_201904.csv',
 'CARD_SUBWAY_MONTH_201905.csv',
 'CARD_SUBWAY_MONTH_201906.csv',
 '지하철노선위경도정보2.csv']

In [84]:
# Google Drive 폴더에 있는 승하차 데이터를 담은 CSV 파일들을 병합하겠습니다.
# 파일 이름에 'CARD'가 포함된 CSV 파일만 선택하여 병합합니다.

print("--- 전체 파일 목록 확인하기 ---")
print(f"Google Drive 폴더 '{drive_dir_path}'의 전체 파일 목록:")
for f in files_in_drive_folder:
    print(f"-", repr(f)) # 파일 이름의 정확한 문자열 표현 출력
print("-------------------")
# --- 디버깅 정보 출력 끝 ---


# 병합할 파일 목록 필터링: 파일 이름에 'CARD'가 포함되고 CSV 확장자인 파일만 선택 -> 지하철노선위경도정보2.csv는 빼기
csv_files_to_merge = [f for f in files_in_drive_folder if f.endswith('.csv') and 'CARD' in f]


# 병합 대상 파일 목록 출력 (수정됨: 실제로 병합될 파일만 출력)
print("\n===== 병합 대상 파일 목록 =====")
if csv_files_to_merge:
    for f in csv_files_to_merge:
        print(f"-", repr(f)) # 병합 대상 파일의 정확한 문자열 표현 출력
else:
    print("병합할 CSV 파일이 없습니다.")
print("============================")

# 빈 데이터프레임 준비
merged_raw = pd.DataFrame()

# 반복문을 통해 필터링된 CSV 파일 읽고 병합
for file_name in csv_files_to_merge:
    file_path = os.path.join(drive_dir_path, file_name)
    # 파일 읽기
    temp = pd.read_csv(file_path)
    # 읽어온 데이터프레임을 기존 데이터프레임에 추가
    merged_raw = pd.concat([merged_raw, temp], ignore_index=True) # 중복 처리 잘 해주기

# 최종 병합된 데이터프레임을 raw 변수에 명확히 다시 할당해주기
raw = merged_raw

display(raw.head())
display(raw.tail())

--- 전체 파일 목록 확인하기 ---
Google Drive 폴더 '/content/drive/MyDrive/AI+X/'의 전체 파일 목록:
- 'CARD_SUBWAY_MONTH_201901.csv'
- 'CARD_SUBWAY_MONTH_201902.csv'
- 'CARD_SUBWAY_MONTH_201903.csv'
- 'CARD_SUBWAY_MONTH_201904.csv'
- 'CARD_SUBWAY_MONTH_201905.csv'
- 'CARD_SUBWAY_MONTH_201906.csv'
- '지하철노선위경도정보2.csv'
-------------------

===== 병합 대상 파일 목록 =====
- 'CARD_SUBWAY_MONTH_201901.csv'
- 'CARD_SUBWAY_MONTH_201902.csv'
- 'CARD_SUBWAY_MONTH_201903.csv'
- 'CARD_SUBWAY_MONTH_201904.csv'
- 'CARD_SUBWAY_MONTH_201905.csv'
- 'CARD_SUBWAY_MONTH_201906.csv'


,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20190101,2호선,을지로4가,3862.0,3728.0,20190104
1,20190101,3호선,을지로3가,8104.0,7554.0,20190104
2,20190101,2호선,을지로입구,22478.0,21330.0,20190104
3,20190101,2호선,시청,8381.0,6049.0,20190104
4,20190101,6호선,동묘앞,8045.0,8504.0,20190104


,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
107023,20190630,7호선,마들,6622.0,6116.0,20190703
107024,20190630,4호선,노원,14071.0,14702.0,20190703
107025,20190630,7호선,중계,9516.0,8981.0,20190703
107026,20190630,7호선,하계,11645.0,11121.0,20190703
107027,20190630,6호선,태릉입구,5730.0,5214.0,20190703


In [85]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107028 entries, 0 to 107027
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   사용일자    107028 non-null  int64  
 1   노선명     107028 non-null  object 
 2   역명      107028 non-null  object 
 3   승차총승객수  107028 non-null  float64
 4   하차총승객수  107028 non-null  float64
 5   등록일자    107028 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 4.9+ MB


In [86]:
raw = raw.reset_index(drop=True)

In [88]:
raw.head(7)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20190101,2호선,을지로4가,3862.0,3728.0,20190104
1,20190101,3호선,을지로3가,8104.0,7554.0,20190104
2,20190101,2호선,을지로입구,22478.0,21330.0,20190104
3,20190101,2호선,시청,8381.0,6049.0,20190104
4,20190101,6호선,동묘앞,8045.0,8504.0,20190104
5,20190101,경의선,청량리,15007.0,15397.0,20190104
6,20190101,1호선,제기동,10187.0,10178.0,20190104


# Q) 요일을 추가할수 있을까?

In [92]:
# 사용일자 컬럼 타입 확인 : int64

raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107028 entries, 0 to 107027
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   사용일자    107028 non-null  int64  
 1   노선명     107028 non-null  object 
 2   역명      107028 non-null  object 
 3   승차총승객수  107028 non-null  float64
 4   하차총승객수  107028 non-null  float64
 5   등록일자    107028 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 4.9+ MB


In [94]:
# '사용일자' 타입 int64 --> DateTime 타입으로 변경 : pandas.to_datetime 사용

raw['사용일자'] = pd.to_datetime(raw['사용일자'], format='%Y%m%d')
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107028 entries, 0 to 107027
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   사용일자    107028 non-null  datetime64[ns]
 1   노선명     107028 non-null  object        
 2   역명      107028 non-null  object        
 3   승차총승객수  107028 non-null  float64       
 4   하차총승객수  107028 non-null  float64       
 5   등록일자    107028 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 4.9+ MB


In [97]:
raw.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,2019-01-01,2호선,을지로4가,3862.0,3728.0,20190104
1,2019-01-01,3호선,을지로3가,8104.0,7554.0,20190104
2,2019-01-01,2호선,을지로입구,22478.0,21330.0,20190104
3,2019-01-01,2호선,시청,8381.0,6049.0,20190104
4,2019-01-01,6호선,동묘앞,8045.0,8504.0,20190104


### pandas.Series.dt
- 참고 사이트 : https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.dayofweek.html
  - pandas.Series.dt.date
  - pandas.Series.dt.time
  - pandas.Series.dt.year
  - pandas.Series.dt.month
  - pandas.Series.dt.day

In [98]:
# yyyy-mm-dd
raw['사용일자'].dt.date.head(3)

,사용일자
0,2019-01-01
1,2019-01-01
2,2019-01-01


In [102]:
# hh-mm-ss
raw['사용일자'].dt.time.head(3)

,사용일자
0,00:00:00
1,00:00:00
2,00:00:00


In [100]:
# yyyy
raw['사용일자'].dt.year.head(3)

,사용일자
0,2019
1,2019
2,2019


In [101]:
# mm
raw['사용일자'].dt.month.head(3)

,사용일자
0,1
1,1
2,1


In [105]:
# dd
raw['사용일자'].dt.day.head(3)

,사용일자
0,1
1,1
2,1


In [106]:
# Monday=0, Sunday=6
raw['사용일자'].dt.dayofweek.head(3)

,사용일자
0,1
1,1
2,1


In [107]:
# 사용일자에 대해서 요일로 변환
raw['요일'] = raw['사용일자'].dt.dayofweek
raw.tail()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,요일
107023,2019-06-30,7호선,마들,6622.0,6116.0,20190703,6
107024,2019-06-30,4호선,노원,14071.0,14702.0,20190703,6
107025,2019-06-30,7호선,중계,9516.0,8981.0,20190703,6
107026,2019-06-30,7호선,하계,11645.0,11121.0,20190703,6
107027,2019-06-30,6호선,태릉입구,5730.0,5214.0,20190703,6


In [108]:
conv = {
    0 : '일',
    1 : '월',
    2 : '화',
    3 : '수',
    4 : '목',
    5 : '금',
    6 : '토'
}

In [109]:
# 숫자 요일을 한글명 요일로 변경
raw['요일'] = raw['요일'].map(conv)


In [121]:
raw.sample(5)

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,요일
7297,2019-01-13,1호선,덕정,4207.0,4637.0,20190116,토
79072,2019-05-14,5호선,고덕,12304.0,12008.0,20190517,월
60987,2019-04-14,3호선,약수,8182.0,7818.0,20190417,토
84862,2019-05-24,경의선,도심,3790.0,2874.0,20190527,목
52001,2019-03-29,경의선,디지털미디어시티,15668.0,13634.0,20190401,목


# Q) '연월' 컬럼을 만들어 보자!

In [129]:
# pandas.Series.dt.strftime 활용

# '사용일자' 컬럼을 'YYYY-MM' 형식으로 변환한 Series 생성 -> series : pandas 1차원 배열
연월_series = raw['사용일자'].dt.strftime('%Y-%m')

# 원본 raw 데이터프레임에서 '노선명' 컬럼을 선택
노선명_series = raw['노선명']

# 두 Series를 합쳐서 새로운 데이터프레임으로 만들고 첫 5행을 표시 -> dataFrame : pandas 2차원 배열
# 인덱스를 기준으로 합쳐지므로 순서가 유지됩니다.
연월_노선명_df = pd.DataFrame({
    '사용일자(YYYY-MM)': 연월_series,
    '노선명': 노선명_series
})

display(연월_노선명_df)

,사용일자(YYYY-MM),노선명
0,2019-01,2호선
1,2019-01,3호선
2,2019-01,2호선
3,2019-01,2호선
4,2019-01,6호선
...,...,...
107023,2019-06,7호선
107024,2019-06,4호선
107025,2019-06,7호선
107026,2019-06,7호선


In [131]:
# 사용일자 컬럼에서 년월 값을 가져와 년월 컬럼 만들자

# DataFrame에서 새로운 컬럼을 추가하거나 기존 컬럼의 내용을 덮어쓸 때
# |  데이터프레임변수['새로운_컬럼_이름'] = 컬럼에_넣을_데이터  | 와 같은 형식을 사용
raw['연월'] = raw['사용일자'].dt.strftime('%Y-%m')
raw.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,요일,연월
0,2019-01-01,2호선,을지로4가,3862.0,3728.0,20190104,월,2019-01
1,2019-01-01,3호선,을지로3가,8104.0,7554.0,20190104,월,2019-01
2,2019-01-01,2호선,을지로입구,22478.0,21330.0,20190104,월,2019-01
3,2019-01-01,2호선,시청,8381.0,6049.0,20190104,월,2019-01
4,2019-01-01,6호선,동묘앞,8045.0,8504.0,20190104,월,2019-01


# Q) '월일' 컬럼을 만들어 보자!

In [132]:
# pandas.Series.dt.strftime 활용
raw['사용일자'].dt.strftime('%m-%d') # m - d

,사용일자
0,01-01
1,01-01
2,01-01
3,01-01
4,01-01
...,...
107023,06-30
107024,06-30
107025,06-30
107026,06-30


In [133]:
# 사용일자 컬럼에서 월일 값을 가져와 월일 컬럼 만들자

raw['월일'] = raw['사용일자'].dt.strftime('%m-%d')
raw.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,요일,연월,월일
0,2019-01-01,2호선,을지로4가,3862.0,3728.0,20190104,월,2019-01,01-01
1,2019-01-01,3호선,을지로3가,8104.0,7554.0,20190104,월,2019-01,01-01
2,2019-01-01,2호선,을지로입구,22478.0,21330.0,20190104,월,2019-01,01-01
3,2019-01-01,2호선,시청,8381.0,6049.0,20190104,월,2019-01,01-01
4,2019-01-01,6호선,동묘앞,8045.0,8504.0,20190104,월,2019-01,01-01


# Q) 승자와 하자하는 모든 승객수를 구해 볼까요?

In [134]:
# 승하자총승객수 컬럼 생성 : 승차총승객수 + 하차총승객수
raw['승하자총승객수'] = raw['승차총승객수'] + raw['하차총승객수']

In [135]:
# 요일을 사용일자 다음에 나오도록 컬럼순서를 변경하겠습니다.

# 현재 컬럼명 확인
print(raw.columns)

raw = raw[['사용일자', '요일', '노선명', '역명', '승차총승객수', '하차총승객수', '승하자총승객수', '연월', '월일', '등록일자' ]]
raw.head()


Index(['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자', '요일', '연월', '월일',
       '승하자총승객수'],
      dtype='object')


,사용일자,요일,노선명,역명,승차총승객수,하차총승객수,승하자총승객수,연월,월일,등록일자
0,2019-01-01,월,2호선,을지로4가,3862.0,3728.0,7590.0,2019-01,01-01,20190104
1,2019-01-01,월,3호선,을지로3가,8104.0,7554.0,15658.0,2019-01,01-01,20190104
2,2019-01-01,월,2호선,을지로입구,22478.0,21330.0,43808.0,2019-01,01-01,20190104
3,2019-01-01,월,2호선,시청,8381.0,6049.0,14430.0,2019-01,01-01,20190104
4,2019-01-01,월,6호선,동묘앞,8045.0,8504.0,16549.0,2019-01,01-01,20190104


# 3. 정리한 데이터 저장하기

In [139]:
# 작업한 결과를 Google Drive에 저장

# 저장할 파일 이름 지정
output_file_name = 'subway_raw.csv'

fpath = os.path.join(drive_dir_path, output_file_name)

# 데이터프레임을 CSV 파일로 저장
# index=False: 데이터프레임의 인덱스를 CSV 파일에 쓰지 않도록 합니다.
# encoding='utf-8-sig': 한글이 깨지지 않도록 인코딩 방식을 지정합니다. (BOM 포함 UTF-8)
try:
    raw.to_csv(fpath, index=False, encoding='utf-8-sig')
    print(f"데이터를 '{fpath}' 경로에 성공적으로 저장했습니다.")
except Exception as e:
    print(f"데이터 저장 중 오류가 발생했습니다: {e}")

데이터를 '/content/drive/MyDrive/AI+X/subway_raw.csv' 경로에 성공적으로 저장했습니다.
